<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/QC_recoding_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:



def purge_wrong_clamp(abf_recordings_df,VC_prot,IC_prot,verbose=False, to_plot=False):
    VC_match_list, IC_match_list = search_wrong_clamp(abf_recordings_df,VC_prot,IC_prot)
    both_good = [VC_match_list[i] and IC_match_list[i] for i in range(len(IC_match_list))]
    abf_recordings_df_purge = abf_recordings_df[both_good]
    both_bad = [not(a) for a in both_good]
    
    bad_file_names = abf_recordings_df.index[both_bad]
    if verbose:
        print('Bad files:')
        _ = [print('     ',f) for f in bad_file_names]
    if to_plot:
        for f in bad_file_names:
            plot_sweeps_and_command(f)

    return abf_recordings_df_purge, bad_file_names

def search_wrong_clamp(abf_recordings_df,VC_prot,IC_prot):
    'takes a df with an index of filename and checks for correct clamping'
    VC_match_list = []
    IC_match_list = []
    for f in abf_recordings_df.index:
        is_VC = clamp_check(f)['is_VC']
        is_VC_prot = abf_recordings_df.loc[f,'protocol'] in VC_prot
        VC_match_list.append(is_VC == is_VC_prot)

        is_IC = clamp_check(f)['is_VC']
        is_IC_prot = abf_recordings_df.loc[f,'protocol'] in IC_prot
        IC_match_list.append(is_IC == is_VC_prot)
    return VC_match_list, IC_match_list



def clamp_check(abf):
    is_IC = False
    is_VC = False
    try:
        abf = abf_or_name(abf)
        if 'mV' in abf.sweepLabelY:
            is_IC = True
            is_VC = False
        if 'pA' in abf.sweepLabelY:
            is_IC = False
            is_VC = True
    except: 
        print('Clamp check error', abf)
    return {'is_IC': is_IC, 'is_VC': is_VC}